In [1]:
from utils.collect_data import keywords

In [103]:
mainquery = {
    "query": {
        "bool": {
            "must": {
                "simple_query_string": {
                    "query": ' '.join(f'({k.replace("-","\-")})' for k in keywords),
                    "fields": [
                        "text",
                        "extended_tweet.full_text"
                    ],
                    # 'minimum_should_match': 1
                }
            },
            "filter": [
                {
                    "range": {
                        "created_at": {
                            "gte": "Tue Nov 01 00:00:00 +0000 2022"
                        }
                    }
                },{
                    "bool": {
                        "must_not": {
                            "exists": {
                                "field": "retweeted_status.id"
                            }
                        },
                        "must": {
                            "exists": {
                                "field": "truncated"
                            }
                        }
                    }
                }
            ]
        }
    }
}

In [106]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import json
from utils.collect_data import prettify_elastic

es_options = json.load(open('./twitter_config.json'))['cache_index']

es = Elasticsearch(
    hosts=[es_options['hostname']],
    verify_certs=False,
    # timeout=config.elasticsearch_timeout_secs
)    

cursor = scan(
        es,
        index=es_options['index_name'],
        query = {**mainquery, '_source':es_options['fields']},
        size=10000
    )

In [104]:
doc_count = es.count(
    index=[es_options['index_name']],
    body=mainquery,
    request_timeout=30
)['count']

In [107]:
texts = [
    c['_source']['extended_tweet']['full_text'] if c['_source']['truncated'] else c['_source']['text']
    for c in tqdm(cursor)
]

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [105]:
doc_count

16625047

In [102]:
doc_count

16625047

In [97]:
doc_count

469176

In [85]:
doc_count

22021503

In [ ]:
doc_count

658382

: 

In [38]:
import json

keywords = json.load(open('utils/keywords.json'))

def has_keyword(text):
    return any(
        text.count(k) > 0
        for k in keywords
    )

In [67]:
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
# load the entire snapshot, apply filter, save the filtered ones in diff directory
snapshot_path = Path('/data/blockchain-interoperability/blockchain-social-media/twitter/snapshots')
# load the entire snapshot, apply filter, save the filtered ones in diff directory
# snapshot_path = Path(snapshot_path)
# huge!
whole_data = pd.concat([
    pd.read_pickle(f)
    for f in tqdm(sorted(snapshot_path.glob('*.pkl')),desc='loading entire dataset..')
])
# good_rows = whole_data[whole_data['whole_text'].apply(has_keyword)]


loading entire dataset..:   0%|          | 0/221 [00:00<?, ?it/s]

In [71]:
from utils.collect_data import has_keyword


subset = whole_data.iloc[:100000]

subset.whole_text.apply(has_keyword).mean()

0.65383

In [49]:
import pickle
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

snapshot_path = Path('/data/blockchain-interoperability/blockchain-social-media/twitter/snapshots/')


for fp in tqdm(sorted(snapshot_path.glob('*.pkl'))):
    df = pd.read_pickle(fp)
    df.drop(columns=['has_keyword'],inplace=True)
    # df['has_keyword'] = df['whole_text'].apply(has_keyword)
    df.to_pickle(fp)

    # break



# all_data = pd.concat([
    
    
# ])
# s = pickle.load(open(00100000.pkl','rb'))

  0%|          | 0/221 [00:00<?, ?it/s]

KeyError: "['has_keyword'] not found in axis"

In [42]:
df.has_keyword.value_counts()

False    56230
True     43770
Name: has_keyword, dtype: int64

In [16]:

    # for k in keywords:
    #     if : 
    #         return True
    # else: return False

In [17]:
all_data['has_keyword'] = all_data.whole_text.apply(has_keyword).mean()

In [19]:
print(f'{all_data.has_keyword.sum().astype(int)}/{all_data.shape[0]} = {all_data.has_keyword.mean()}')

11340722/22021502 = 0.5149840369653367
